In [4]:
import pandas as pd
import json
import os
import gzip
import numpy as np

In [22]:
root_dir = os.getcwd().split("RAG\\data\\GSEA")[0] + "RAG"
target_dir = os.path.join(root_dir, "data", "GSEA")
if os.path.exists(target_dir):
    os.chdir(target_dir)
else:
    print(f"Error: The directory '{target_dir}' does not exist.")
print("Current working directory:", os.getcwd())

Current working directory: C:\Python\RAG\Data\GSEA


In [ ]:
file = "rat_data.txt.gz"
input_path = os.path.join("to_be_converted", file)
output_path = os.path.join("to_be_converted", f"reordered_{file}")

# Ensure the input file exists
if not os.path.exists(input_path):
    print(f"Error: File '{input_path}' does not exist.")
else:
    # Define the columns to extract
    cols = [
        "Gene stable ID",
        "Gene name",
        "Gene description",
        "Gene Synonym",
        "NCBI gene (formerly Entrezgene) description",
        "NCBI gene (formerly Entrezgene) ID"
    ]
    
    try:
        # Read the gzipped CSV file
        df = pd.read_csv(input_path, compression="gzip")
    except Exception as e:
        print("Error reading input file:", e)
    else:
        # Check if all required columns exist
        missing_cols = [col for col in cols if col not in df.columns]
        if missing_cols:
            print("Error: The following required columns are missing:", missing_cols)
        else:
            # Select and reorder the desired columns
            df_selected = df[cols]
            
            try:
                # Write the output to a new gzipped CSV file
                df_selected.to_csv(output_path, index=False, compression="gzip")
                print(f"Reordered file saved as {output_path}")
            except Exception as e:
                print("Error writing output file:", e)


In [ ]:
os.chdir('../..')

In [ ]:
print(os.getcwd())

In [ ]:
Add_Synonyms = True

In [ ]:

input_file = './data/GSEA/to_be_converted/wikipathways-20241210-gmt-Rattus_norvegicus.gmt'
output_file = './data/GSEA/to_be_converted/converted_wikipathways-20241210-gmt-Rattus_norvegicus.gmt'

json_file = './data/JSON/ncbi_id_to_symbol.json'

# Load the gene ID to symbol dictionary from JSON
with open(json_file, 'r') as f:
    gene_dict = json.load(f)

# Read the GMT input file
columns = ['header', 'url'] + [f'gene_{i}' for i in range(1000)]  
df = pd.read_csv(input_file, sep='\t', header=None, names=columns, engine='python', dtype=str, na_filter=False)

# Function to replace gene IDs with symbols
def replace_gene_ids(gene_id):
    return gene_dict.get(gene_id, gene_id)

# Apply replacement function to gene columns
gene_columns = df.columns[2:]
for col in gene_columns:
    df[col] = df[col].apply(replace_gene_ids)

# Remove empty columns (if all values in a column are empty strings)
df = df.loc[:, (df != '').any(axis=0)]

# Remove empty rows for gene columns to prevent excess line breaks
df = df.apply(lambda x: x.dropna().tolist(), axis=1).apply(pd.Series)

# Save to the output file without excess newlines
df.to_csv(output_file, sep='\t', header=False, index=False, lineterminator='\n')

print(f'File conversion completed! Output saved to {output_file}')


In [ ]:
input_file = './data/GSEA/to_be_converted/reordered_rat_data.txt.gz'  
output_json = './data/JSON/genes.json' 
output_file = './data/GSEA/external_gene_data/rat_genes_consolidated.txt.gz' #remove rat

In [ ]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

ensure_dir(output_json)
ensure_dir(output_file)

df = pd.read_csv(input_file, compression='gzip')
print(f"Successfully read input file: {input_file}")

df['Gene Synonym'] = df['Gene Synonym'].fillna('').astype(str).str.strip()

df['Gene stable ID'] = df['Gene stable ID'].str.strip().str.upper()

unique_gene_ids = df['Gene stable ID'].nunique()
print(f"Number of unique 'Gene stable ID's: {unique_gene_ids}")
# Grouping the data and applying transformations
grouped = df.groupby('Gene stable ID').agg({
    'Gene name': 'first',
    'Gene description': 'first',
    'Gene Synonym': lambda x: sorted(filter(None, x.unique())),
    'NCBI gene (formerly Entrezgene) description': 'first',
    'NCBI gene (formerly Entrezgene) ID': 'first'
}).reset_index()
# Ensure 'Gene Synonyms' are formatted correctly
grouped.rename(columns={'Gene Synonym': 'Gene Synonyms'}, inplace=True)
grouped['Gene Synonyms'] = grouped['Gene Synonyms'].apply(lambda x: f"[{','.join(x)}]" if x else "[]")

# 1. Removing NCBI description unless Gene description is empty and NCBI description is not
grouped['Gene description'] = grouped.apply(
    lambda row: row['NCBI gene (formerly Entrezgene) description']
    if pd.isna(row['Gene description']) and pd.notna(row['NCBI gene (formerly Entrezgene) description'])
    else row['Gene description'],
    axis=1
)

# Removing descriptions between []. 
grouped['Gene description'] = grouped['Gene description'].str.replace(r'\[.*?\]', '', regex=True).str.strip()

#Dropping the NCBI description column as it's no longer needed, and converting the ID to integer.
grouped.drop(columns=['NCBI gene (formerly Entrezgene) description'], inplace=True)
grouped['NCBI gene (formerly Entrezgene) ID'] = grouped['NCBI gene (formerly Entrezgene) ID'].astype('Int64')

consolidated_entries = grouped.shape[0]
print(f"Number of consolidated entries: {consolidated_entries}")

if consolidated_entries != unique_gene_ids:
    print("Warning: The number of consolidated entries does not match the number of unique 'Gene stable ID's.")
    print(f"Unique 'Gene stable ID's: {unique_gene_ids}, Consolidated entries: {consolidated_entries}")
else:
    print("Success: The number of consolidated entries matches the number of unique 'Gene stable ID's.")

genes_list = grouped.to_dict(orient='records')

with open(output_json, 'w', encoding='utf-8') as f_json:
    json.dump(genes_list, f_json, indent=4)
print(f"Consolidated JSON data has been saved to '{output_json}'.")

ordered_columns = [
    'Gene stable ID',
    'Gene name',
    'Gene description',
    #'Gene Synonyms',
    #'NCBI gene (formerly Entrezgene) ID'
]

missing_columns = set(ordered_columns) - set(grouped.columns)
if missing_columns:
    print(f"Error: Missing columns in the DataFrame: {missing_columns}")
    exit(1)

grouped_ordered = grouped[ordered_columns]

grouped_ordered.to_csv(output_file, index=False, sep=',', compression='gzip')
print(f"Consolidated TXT.GZ data has been saved to '{output_file}'.")


In [ ]:
genes_json_path = './data/JSON/genes.json'  
input_gmt_path = './data/GSEA/to_be_converted/converted_wikipathways-20241210-gmt-Rattus_norvegicus.gmt'
output_gmt_path = './data/GSEA/external_gene_data/wikipathways_synonyms_Rattus_norvegicus.gmt.gz'

In [ ]:
ensure_dir(output_gmt_path)

# Load genes data
with open(genes_json_path, 'r', encoding='utf-8') as f_json:
    genes_data = json.load(f_json)

# Create gene to synonyms mapping
gene_to_synonyms = {}
for entry in genes_data:
    gene_name = entry.get('Gene name')
    if gene_name is None:
        continue
    gene_name = gene_name.strip()
    
    synonyms_str = entry.get('Gene Synonyms') or ''
    synonyms_str = synonyms_str.strip()
    if synonyms_str.startswith('[') and synonyms_str.endswith(']'):
        synonyms_str = synonyms_str[1:-1]
    
    synonyms = synonyms_str.split('_') if synonyms_str else []
    synonyms = [syn.strip() for syn in synonyms if syn.strip()]
    
    gene_to_synonyms[gene_name] = synonyms

print(f"Loaded {len(gene_to_synonyms)} genes with synonyms.")

# Process GMT file and save as .gmt.gz
def process_gmt(input_path, output_path, gene_synonyms_map):
    base_url = "https://www.wikipathways.org/instance/"
    with open(input_path, 'r', encoding='utf-8') as infile, gzip.open(output_path, 'wt', encoding='utf-8') as outfile:
        for line_number, line in enumerate(infile, 1):
            line = line.strip()
            if not line:
                continue  
            parts = line.split('\t')
            if len(parts) < 3:
                print(f"Warning: Line {line_number} in GMT file does not have enough columns. Skipping.")
                continue
            pathway_name_full, pathway_url_full, *genes = parts
            
            if '%' in pathway_name_full:
                pathway_name = pathway_name_full.split('%')[0].strip()
            else:
                pathway_name = pathway_name_full.strip()
            
            if pathway_url_full.startswith(base_url):
                pathway_url = pathway_url_full.replace(base_url, '').strip()
            else:
                pathway_url = pathway_url_full.strip()
            
            expanded_genes = []
            for gene in genes:
                gene = gene.strip()
                if not gene:
                    continue  
                synonyms = gene_synonyms_map.get(gene, [])
                if synonyms:
                    expanded_gene = f"[{gene}, " + ", ".join(synonyms) + "]"
                else:
                    expanded_gene = f"[{gene}]"
                expanded_genes.append(expanded_gene)
            
            # **4. Remove duplicate genes while preserving order**
            seen = set()
            unique_genes = []
            for gene in expanded_genes:
                if gene not in seen:
                    seen.add(gene)
                    unique_genes.append(gene)
            
            # **5. Assemble the new line**
            new_line = '\t'.join([pathway_name, pathway_url] + unique_genes)
            outfile.write(new_line + '\n')
            
            # **6. Progress Logging**
            if line_number % 1000 == 0:
                print(f"Processed {line_number} lines.")
    print(f"Finished processing GMT file. Output saved to '{output_path}'.")

process_gmt(input_gmt_path, output_gmt_path, gene_to_synonyms)

In [ ]:
# Import necessary libraries
import pandas as pd
import gzip
import csv
import os
import time


def initialize_gene_list(excel_file_path=r".\data\GSEA\genes_of_interest\PMP22_VS_WT.xlsx", de_filter_option="combined", test=False):
    """
    Initializes the gene list by processing the Excel file.

    Parameters:
    - excel_file_path: Path to the Excel file containing gene data.
    - de_filter_option: Filter option ("combined" or "separate").
    - test: Boolean flag for test mode.

    Returns:
    - gene_list_string: Comma-separated string of gene names.
    - regulation: Regulation type ("upregulated", "downregulated", "combined").
    - num_genes: Number of genes in the list.
    """
    results = process_excel_data(excel_file_path, de_filter_option, test)

    if results:
        gene_list_string, regulation, num_genes = results[0]
    else:
        gene_list_string = ""
        regulation = ""
        num_genes = 0

    return gene_list_string, regulation, num_genes


def process_excel_data(excel_file_path, de_filter_option, test):
    """
    Processes the Excel data to filter genes based on DE and FDR thresholds.

    Parameters:
    - excel_file_path: Path to the Excel file.
    - de_filter_option: Filter option ("combined" or "separate").
    - test: Boolean flag for test mode.

    Returns:
    - results: List of tuples containing gene list string, regulation, and number of genes.
    """
    data = pd.read_excel(excel_file_path)
    results = []
    fdr_threshold = 0.00008802967327

    if not test:
        max_genes = 250
        data = data.iloc[:max_genes]

        if de_filter_option == "combined":
            data = data[data['DE'] != 0]
            if not data.empty and data['FDR'].max() <= fdr_threshold:
                genes_list = data['X'].tolist()
                num_genes = len(genes_list)
                unique_de_values = data['DE'].unique()

                regulation = (
                    "upregulated" if len(unique_de_values) == 1 and unique_de_values[0] == 1 else
                    "downregulated" if len(unique_de_values) == 1 and unique_de_values[0] == -1 else
                    "combined"
                )
                results.append((', '.join(genes_list), regulation, num_genes))

        elif de_filter_option == "separate":
            for de_value, regulation_label in [(1, "upregulated"), (-1, "downregulated")]:
                filtered_data = data[data['DE'] == de_value]
                if not filtered_data.empty and filtered_data['FDR'].max() <= fdr_threshold:
                    genes_list = filtered_data['X'].tolist()
                    num_genes = len(genes_list)
                    results.append((', '.join(genes_list), regulation_label, num_genes))

        else:
            raise ValueError("Invalid DE filter option. Use 'combined' or 'separate'.")

    else:
        # Handling for test=True can be implemented here if needed
        pass

    return results


def extract_gene_descriptions(gene_list_string, gene_data_file='rat_genes_consolidated.txt.gz', output_csv='gene_descriptions.csv'):

    if not gene_list_string:
        print("Gene list is empty. No descriptions to extract.")
        return

    # Step 1: Parse the gene list string into a list
    gene_names = [gene.strip() for gene in gene_list_string.split(',') if gene.strip()]
    gene_names_set = set(gene_names)  # For faster lookup

    print(f"Total genes to process: {len(gene_names_set)}")
    print(gene_names_set)
    # Step 2: Extract gene descriptions from the compressed file
    gene_description_dict = {}

    if not os.path.exists(gene_data_file):
        print(f"Gene data file '{gene_data_file}' does not exist.")
        return

    try:
        with gzip.open(gene_data_file, 'rt', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                gene_name = row.get('Gene name', '').strip()
                if gene_name in gene_names_set:
                    description = row.get('Gene description', '').strip()
                    gene_description_dict[gene_name] = description
                    # Optionally remove found genes to speed up
                    gene_names_set.remove(gene_name)
                    if not gene_names_set:
                        break  # All genes found
    except Exception as e:
        print(f"Error processing the gene data file: {e}")
        return

    # Step 3: Create a DataFrame for the output
    output_data = {
        'Gene name': [],
        'Gene description': []
    }

    for gene in gene_names:
        description = gene_description_dict.get(gene, "Description not found")
        output_data['Gene name'].append(gene)
        output_data['Gene description'].append(description)

    output_df = pd.DataFrame(output_data)

    # Step 4: Save the output to a new CSV file
    try:
        output_df.to_csv(output_csv, index=False)
        print(f"Successfully created '{output_csv}' with {len(output_df)} entries.")
    except Exception as e:
        print(f"Error writing the output CSV file: {e}")

    # Optional: Report missing genes
    missing_genes = [gene for gene in gene_names if gene not in gene_description_dict]
    if missing_genes:
        print(f"The following genes were not found in the gene data file: {', '.join(missing_genes)}")

# Define file paths
excel_file_path = r".\data\GSEA\genes_of_interest\PMP22_VS_WT.xlsx"  # Path to your Excel file
gene_data_file = r'.\data\GSEA\external_gene_data\rat_genes_consolidated.txt.gz'  # Path to the compressed gene data file
output_csv = r'.\data\GSEA\external_gene_data\gene_descriptions.csv'  # Desired output CSV file name

# Initialize the gene list
gene_list_string, regulation, num_genes = initialize_gene_list(
    excel_file_path=excel_file_path,
    de_filter_option="combined",  # Change to "separate" if needed
    test=False  # Set to True for test mode
)

# Extract gene descriptions and save to CSV
extract_gene_descriptions(
    gene_list_string=gene_list_string,
    gene_data_file=gene_data_file,
    output_csv=output_csv
)
